In [ ]:
!pip install -U bitsandbytes

In [ ]:
import pandas as pd

In [ ]:
news= pd.read_excel('/content/summarized_content (1).xlsx')

In [ ]:
news.drop('Content', axis=1, inplace=True)

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install torch transformers


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = 'EleutherAI/gpt-j-6B'
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/930 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


pytorch_model.bin:   0%|          | 0.00/24.2G [00:00<?, ?B/s]

Some weights of the model checkpoint at EleutherAI/gpt-j-6B were not used when initializing GPTJForCausalLM: ['transformer.h.0.attn.bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.10.attn.bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.12.attn.bias', 'transformer.h.12.attn.masked_bias', 'transformer.h.13.attn.bias', 'transformer.h.13.attn.masked_bias', 'transformer.h.14.attn.bias', 'transformer.h.14.attn.masked_bias', 'transformer.h.15.attn.bias', 'transformer.h.15.attn.masked_bias', 'transformer.h.16.attn.bias', 'transformer.h.16.attn.masked_bias', 'transformer.h.17.attn.bias', 'transformer.h.17.attn.masked_bias', 'transformer.h.18.attn.bias', 'transformer.h.18.attn.masked_bias', 'transformer.h.19.attn.bias', 'transformer.h.19.attn.masked_bias', 'transformer.h.2.attn.bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.20.attn.bi

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

In [ ]:
text= news['Content_Summary']

In [ ]:
def get_sentiment(text):
    prompt = f"""
    Text: Global demand for iron ore has surged, leading to higher prices.
    Sentiment: increase

    Text: The iron ore market faced a downturn due to declining demand in key markets.
    Sentiment: decrease

    Text: Prices have dropped as global trade tensions have disrupted iron ore exports.
    Sentiment: decrease

    Text: {text}
    Sentiment:
    """

    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).input_ids.to("cuda" if torch.cuda.is_available() else "cpu")

    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        temperature=0.7,
        max_length=input_ids.shape[1] + 5,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    print(f"Generated text: {generated_text}")  # Çıktıyı kontrol etmek için ekle

    sentiment = generated_text.split("Sentiment:")[-1].strip()
    sentiment = sentiment.lower()

    if sentiment not in ["increase", "decrease", "neutral"]:
        sentiment = "neutral"

    return sentiment



In [ ]:
news['Sentiment'] = news['Content_Summary'].apply(get_sentiment)


Generated text: 
    Text: Global demand for iron ore has surged, leading to higher prices.
    Sentiment: increase

    Text: The iron ore market faced a downturn due to declining demand in key markets.
    Sentiment: decrease

    Text: Prices have dropped as global trade tensions have disrupted iron ore exports.
    Sentiment: decrease

    Text: iron ore futures on major Asian exchanges have been declining . the decline reflects a slowdown in economic activity and a general decline in demand for raw materials caused by weak economic performance in China .
    Sentiment:
        

Generated text: 
    Text: Global demand for iron ore has surged, leading to higher prices.
    Sentiment: increase

    Text: The iron ore market faced a downturn due to declining demand in key markets.
    Sentiment: decrease

    Text: Prices have dropped as global trade tensions have disrupted iron ore exports.
    Sentiment: decrease

    Text: iron ore futures are the most traded on the dalian Commod

In [ ]:
news.head()

,Date,Content_Summary,Sentiment
0,12 November 2024,iron ore futures on major Asian exchanges have...,neutral
1,29 October 2024,iron ore futures are the most traded on the da...,neutral
2,16 October 2024,iron ore futures are the most traded on the da...,neutral
3,08 October 2024,prices for raw materials continued to move upw...,neutral
4,30 September 2024,iron ore prices rose by more than 7% in one da...,neutral


In [ ]:
news

,Date,Content_Summary,Sentiment
0,12 November 2024,iron ore futures on major Asian exchanges have...,neutral
1,29 October 2024,iron ore futures are the most traded on the da...,neutral
2,16 October 2024,iron ore futures are the most traded on the da...,neutral
3,08 October 2024,prices for raw materials continued to move upw...,neutral
4,30 September 2024,iron ore prices rose by more than 7% in one da...,neutral
...,...,...,...
115,29 June 2022,world iron ore production will increase by an ...,neutral
116,28 June 2022,prices for iron ore have fallen since the begi...,neutral
117,28 June 2022,a sharp drop of steel raw materials prices mak...,neutral
118,27 June 2022,the cost of iron ore on the dalian Commodity E...,neutral


In [ ]:
news.to_excel('iron_ore_sentiment_classified.xlsx', index=False)
